In [2]:
import pandas as pd
from scipy.io import arff
import numpy
import math
import operator

In [3]:
data=arff.loadarff("./product-selection/trainProdSelection.arff")
trainDF = pd.DataFrame(data[0])
data=arff.loadarff("./product-selection/testProdSelection.arff")
testDF = pd.DataFrame(data[0])

In [4]:
list(pd.DataFrame(data[0]))
pd.DataFrame(data[0]).dtypes

Type          object
LifeStyle     object
Vacation     float64
eCredit      float64
salary       float64
property     float64
label         object
dtype: object

In [5]:
trainDF.Type=trainDF.Type.str.decode("UTF-8")

In [6]:
trainDF.LifeStyle=trainDF.LifeStyle.str.decode("UTF-8")

In [7]:


trainDF.label=trainDF.label.str.decode("UTF-8")

In [8]:
minValue=trainDF.Vacation.min()
maxValue=trainDF.Vacation.max()
trainDF.Vacation=trainDF.Vacation.apply(lambda x:(x-minValue)/(maxValue-minValue))


In [9]:
minValue=trainDF.eCredit.min()
maxValue=trainDF.eCredit.max()
trainDF.eCredit=trainDF.eCredit.apply(lambda x:(x-minValue)/(maxValue-minValue))

In [10]:
minValue=trainDF.salary.min()
maxValue=trainDF.salary.max()
trainDF.salary=trainDF.salary.apply(lambda x:(x-minValue)/(maxValue-minValue))

minValue=trainDF.property.min()
maxValue=trainDF.property.max()
trainDF.property=trainDF.property.apply(lambda x:(x-minValue)/(maxValue-minValue))

In [11]:
trainDF.head()

,Type,LifeStyle,Vacation,eCredit,salary,property,label
0,student,spend>saving,0.079365,0.107558,0.219960,0.183167,C1
1,student,spend>saving,0.158730,0.052326,0.293102,0.112797,C1
2,student,spend>saving,0.095238,0.177326,0.346023,0.174200,C1
3,student,spend>saving,0.031746,0.127907,0.309882,0.189984,C1
4,student,spend>saving,0.222222,0.020349,0.363663,0.127311,C1


In [12]:
m=trainDF.Vacation.mean()
stddev=numpy.std(trainDF.Vacation)
trainDF.Vacation=trainDF.Vacation.apply(lambda x:(x-m)/(stddev))
m=trainDF.eCredit.mean()
stddev=numpy.std(trainDF.eCredit)
trainDF.eCredit=trainDF.eCredit.apply(lambda x:(x-m)/(stddev))
m=trainDF.salary.mean()
stddev=numpy.std(trainDF.salary)
trainDF.salary=trainDF.salary.apply(lambda x:(x-m)/(stddev))
m=trainDF.property.mean()
stddev=numpy.std(trainDF.property)
trainDF.property=trainDF.property.apply(lambda x:(x-m)/(stddev))

In [13]:
testDF.Type=testDF.Type.str.decode("UTF-8")

In [14]:
testDF.LifeStyle=testDF.LifeStyle.str.decode("UTF-8")

In [15]:
testDF.label=testDF.label.str.decode("UTF-8")

In [16]:
minValue=testDF.Vacation.min()
maxValue=testDF.Vacation.max()
testDF.Vacation=testDF.Vacation.apply(lambda x:(x-minValue)/(maxValue-minValue))


In [17]:
minValue=testDF.eCredit.min()
maxValue=testDF.eCredit.max()
testDF.eCredit=testDF.eCredit.apply(lambda x:(x-minValue)/(maxValue-minValue))

In [18]:
minValue=testDF.salary.min()
maxValue=testDF.salary.max()
testDF.salary=testDF.salary.apply(lambda x:(x-minValue)/(maxValue-minValue))

In [19]:
minValue=testDF.property.min()
maxValue=testDF.property.max()
testDF.property=testDF.property.apply(lambda x:(x-minValue)/(maxValue-minValue))

In [20]:
testDF.head()

,Type,LifeStyle,Vacation,eCredit,salary,property,label
0,student,spend<saving,0.20,0.058824,0.104637,0.398926,C1
1,student,spend>>saving,0.54,0.021008,0.175059,0.243041,C1
2,student,spend<<saving,0.52,0.231092,0.138339,0.085992,C1
3,engineer,spend>saving,0.26,0.151261,0.430086,0.116229,C1
4,librarian,spend<saving,0.00,0.016807,0.352657,0.025714,C1


In [21]:
m=testDF.Vacation.mean()
stddev=numpy.std(testDF.Vacation)
testDF.Vacation=testDF.Vacation.apply(lambda x:(x-m)/(stddev))
m=testDF.eCredit.mean()
stddev=numpy.std(testDF.eCredit)
testDF.eCredit=testDF.eCredit.apply(lambda x:(x-m)/(stddev))
m=testDF.salary.mean()
stddev=numpy.std(testDF.salary)
testDF.salary=testDF.salary.apply(lambda x:(x-m)/(stddev))
m=testDF.property.mean()
stddev=numpy.std(testDF.property)
testDF.property=testDF.property.apply(lambda x:(x-m)/(stddev))

In [22]:
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for i in range(2):
        if (instance1[i]==instance2[i]):
            distance += pow((1), 2)
    for x in range(2,length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)
 
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors
 
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]
 
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

#def getMinMax(trainSet):
 #   l=[]
  #  c=0
   # for i in list(trainSet):
    #    if trainSet[i].dtype=='float64':
     #       l.append((trainSet[i].min(),trainSet[i].max()))
      #  else:
       #     l.append((0,0))
    #return l

In [23]:
def knn(k):
    predictions=[]
    for x in range(len(testDF)):
        neighbors = getNeighbors(trainDF.values, testDF.values[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        print('> predicted=' + repr(result) + ', actual=' + repr(testDF.values[x][-1]))
    accuracy = getAccuracy(testDF.values, predictions)
    print('Accuracy: ' + repr(accuracy) + '%')

In [24]:
#getMinMax(trainDF)
#list(trainDF)

In [25]:
knn(3)

> predicted='C1', actual='C1'
> predicted='C4', actual='C1'
> predicted='C4', actual='C1'
> predicted='C4', actual='C1'
> predicted='C2', actual='C1'
> predicted='C1', actual='C1'
> predicted='C2', actual='C1'
> predicted='C2', actual='C1'
> predicted='C3', actual='C1'
> predicted='C3', actual='C1'
> predicted='C3', actual='C1'
> predicted='C3', actual='C1'
> predicted='C5', actual='C1'
> predicted='C1', actual='C1'
> predicted='C4', actual='C1'
> predicted='C1', actual='C1'
> predicted='C4', actual='C1'
> predicted='C4', actual='C1'
> predicted='C5', actual='C1'
> predicted='C5', actual='C1'
> predicted='C4', actual='C1'
Accuracy: 19.047619047619047%


In [26]:
TotalDF=pd.concat([trainDF, testDF], ignore_index=True)

In [27]:
TotalDF

,Type,LifeStyle,Vacation,eCredit,salary,property,label
0,student,spend>saving,-1.163684,-0.329532,-1.656259,-0.229398,C1
1,student,spend>saving,-0.894004,-0.605823,-1.256581,-0.564126,C1
2,student,spend>saving,-1.109748,0.019467,-0.967402,-0.272051,C1
3,student,spend>saving,-1.325492,-0.227741,-1.164890,-0.196969,C1
4,student,spend>saving,-0.678260,-0.765781,-0.871009,-0.495087,C1
5,student,spend>saving,-1.163684,0.252133,-1.214262,-0.108681,C1
6,student,spend<saving,-0.947940,-0.198657,-0.894520,0.458456,C1
7,student,spend<saving,-0.947940,0.310300,-1.411750,-0.254266,C1
8,student,spend<saving,-1.001876,0.164883,-1.599834,-0.468516,C1
9,student,spend>>saving,-0.300708,-0.358615,-1.595131,-0.905936,C1


In [28]:
from sklearn.model_selection import train_test_split

In [29]:
minValue=TotalDF.Vacation.min()
maxValue=TotalDF.Vacation.max()
TotalDF.Vacation=TotalDF.Vacation.apply(lambda x:(x-minValue)/(maxValue-minValue))


In [30]:
minValue=TotalDF.eCredit.min()
maxValue=TotalDF.eCredit.max()
TotalDF.eCredit=TotalDF.eCredit.apply(lambda x:(x-minValue)/(maxValue-minValue))


In [31]:
minValue=TotalDF.salary.min()
maxValue=TotalDF.salary.max()
TotalDF.salary=TotalDF.salary.apply(lambda x:(x-minValue)/(maxValue-minValue))


In [32]:
minValue=TotalDF.property.min()
maxValue=TotalDF.property.max()
TotalDF.property=TotalDF.property.apply(lambda x:(x-minValue)/(maxValue-minValue))

In [33]:
x=TotalDF[list(TotalDF)[0:6]].values

In [34]:
y=TotalDF[list(TotalDF)[-1]].values

In [35]:
train_x,test_x,train_y,train_y=train_test_split(x,y,test_size=21/TotalDF.shape[0])

In [36]:
train_x.shape,test_x.shape,train_y.shape,train_y.shape

((186, 6), (21, 6), (21,), (21,))

In [37]:
train_x[:5]

array([['professor', 'spend>saving', 0.761904761904762,
        0.011627906976744184, 0.8747977833614428, 0.23701824273796382],
       ['doctor', 'spend>saving', 0.14285714285714285,
        0.6366279069767442, 0.7384091143771728, 0.8509428108562304],
       ['professor', 'spend<saving', 0.07936507936507935,
        0.04069767441860467, 0.43398272123360754, 0.06318582069920677],
       ['professor', 'spend>>saving', 0.7936507936507936,
        0.008720930232558138, 0.7853061645957388, 0.15080942548650325],
       ['engineer', 'spend>>saving', 0.7777777777777777,
        0.1511627906976744, 0.5297387533129109, 0.18556494254901562]],
      dtype=object)

In [38]:
trainDF=TotalDF.iloc[:186,:]
testDF=TotalDF.iloc[186:,:]
testDF.shape
#trainDF=pd.DataFrame(data=trainDF,index=range(0,len(trainDF)),columns=list(['Type', 'LifeStyle', 'Vacation', 'eCredit', 'salary', 'property', 'label'])) 
#testDF=pd.DataFrame(data=testDF,index=range(0,len(testDF)),columns=list(['Type', 'LifeStyle', 'Vacation', 'eCredit', 'salary', 'property', 'label'])) 

(21, 7)

In [39]:
TotalDF

,Type,LifeStyle,Vacation,eCredit,salary,property,label
0,student,spend>saving,0.079365,0.107558,0.219960,0.199751,C1
1,student,spend>saving,0.158730,0.052326,0.293102,0.130810,C1
2,student,spend>saving,0.095238,0.177326,0.346023,0.190966,C1
3,student,spend>saving,0.031746,0.127907,0.309882,0.206430,C1
4,student,spend>saving,0.222222,0.020349,0.363663,0.145030,C1
5,student,spend>saving,0.079365,0.223837,0.300847,0.224614,C1
6,student,spend<saving,0.142857,0.133721,0.359360,0.341422,C1
7,student,spend<saving,0.142857,0.235465,0.264706,0.194629,C1
8,student,spend<saving,0.126984,0.206395,0.230286,0.150502,C1
9,student,spend>>saving,0.333333,0.101744,0.231147,0.060411,C1


In [43]:
knn(5)

> predicted='C1', actual='C1'
> predicted='C4', actual='C1'
> predicted='C4', actual='C1'
> predicted='C4', actual='C1'
> predicted='C2', actual='C1'
> predicted='C1', actual='C1'
> predicted='C2', actual='C1'
> predicted='C2', actual='C1'
> predicted='C1', actual='C1'
> predicted='C4', actual='C1'
> predicted='C3', actual='C1'
> predicted='C3', actual='C1'
> predicted='C4', actual='C1'
> predicted='C1', actual='C1'
> predicted='C4', actual='C1'
> predicted='C1', actual='C1'
> predicted='C4', actual='C1'
> predicted='C4', actual='C1'
> predicted='C5', actual='C1'
> predicted='C5', actual='C1'
> predicted='C4', actual='C1'
Accuracy: 23.809523809523807%


In [41]:
# creating list of K for KNN
k_list = list(range(1,50,2))
# creating list of cv scores
cv_scores = []
from sklearn.model_selection import cross_val_score
# perform 10-fold cross validation
for k in k_list:
    knn1 = knn(k)
    scores = cross_val_score(knn1, train_x, train_y, cv=10, scoring='accuracy')
    cv_scores.append(scores)

> predicted='C1', actual='C1'
> predicted='C4', actual='C1'
> predicted='C4', actual='C1'
> predicted='C4', actual='C1'
> predicted='C2', actual='C1'
> predicted='C2', actual='C1'
> predicted='C2', actual='C1'
> predicted='C2', actual='C1'
> predicted='C1', actual='C1'
> predicted='C4', actual='C1'
> predicted='C3', actual='C1'
> predicted='C1', actual='C1'
> predicted='C4', actual='C1'
> predicted='C1', actual='C1'
> predicted='C4', actual='C1'
> predicted='C1', actual='C1'
> predicted='C4', actual='C1'
> predicted='C4', actual='C1'
> predicted='C5', actual='C1'
> predicted='C5', actual='C1'
> predicted='C4', actual='C1'
Accuracy: 23.809523809523807%


TypeError: estimator should be an estimator implementing 'fit' method, None was passed